This file details our efforts to compare a variety of different prompts and parameters in the use of a pretrained model to conduct few shot learning and generate a fake profile that sounds indistinguishable from other top matches.

Different avenues explored include using:
    - Full essays 
        - Example prompt:
            Input: 
            Output:
            Input:
            Output:
            Input:
            
    - First x number of characters of an essay
            Example prompt: Same as above but cut off each essay after x characters
            
    - Good and bad match
        - Example prompt:
            This is the input essay: 
            This is a good match:
            This is a bad match: 
            Write a new good match:
            
    - Experiment with different parameters of the generate function
    
    - different/larger versions of GPT - implement when we have more computational resources



**NEED TO UPDATE This file loads in a pretrained model from the HuggingFace library, and uses most of an individuals essays and a csv containing the indices of their top match, to construct a prompt for a few shot encoder to produce a fake profile in the same style/tone that should also be a good match for them.




In [144]:
from transformers import pipeline
import pandas as pd

generator = pipeline('text-generation', model='distilgpt2') #CONSIDER USING FULL GPT-2 OR GPT-2-SMALL IN CLOUD

generated_text = generator("EleutherAI has", max_length=50, num_return_sequences=1, truncation = True)

print(generated_text[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


EleutherAI has released a major new study based on a series of studies showing that the species has been affected by extinction for many generations. This study from the University of Colorado and the University of Massachusetts-Dade, U.S., shows that


In [40]:
#Loading in the csv with the top five people that are matches for our person

top_indices = pd.read_csv('top5_indices.csv').set_index('Unnamed: 0')
top_indices.columns = ['match_idx']

In [41]:
top_indices

,match_idx
Unnamed: 0,
0,4
1,8
2,9
3,2
4,5


In [61]:
df = pd.read_csv("classifier/classifier_outputs.csv").set_index('Unnamed: 0')

In [133]:
#Build the prompt that we're going to use for few shot encoding with our model
#NEEDS TO BE UPDATED, include a max length functionality
def construct_prompt(df, top_matches, input_prompt):
    top_subset = df.loc[top_indices.loc[:2,'match_idx'],'essay0']
    prompt = "Write a completely new dating profile that is a good match for the input profile "
    for match in top_subset: 
        if type(match) is float: 
             continue
        prompt = prompt + "\n " + "Input: " + input_prompt + "\n Output: " + match
    prompt = prompt + "\n " + "Input: " + input_prompt + "\n" + " Output: "
    return prompt
essay_prompt = construct_prompt(df , top_indices, df.loc[6,'essay0'])
print(essay_prompt)  

Write a completely new dating profile that is a good match for the input profile 
 Input: i keep lots of pens and paper around. i enjoy virtually anything athletic (rock climbing, hiking, sports,...) except running for running's sake, which i respect but do not enjoy (except when involved in not-running). i have no respect for people who can't change their opinions in the face of new evidence. i do not believe that the real numbers really exist because i do not believe in the power-set axiom of set theory for infinite sets. whatever that means. and i'm from texas but don't hold it against me.
 Output: the quintessential san francisco experience is to randomly meet strangers and party on their roof. i have two other friends from ohio move out here and they've had that same experience each within the first month of living here. it's that kind of random cool stuff that makes me stay in this city and doesn't allow me to entertain thoughts of moving back to the stripmall paradise of boredom

In [142]:

all_returned = generator(essay_prompt, do_sample=True, temperature = 0.9,
                         min_length=50, max_length = 1024, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [143]:
#This is just the generated essay. 
all_returned[0]['generated_text'].replace(essay_prompt, "")

"iphone x 1 + 1\nI've come to the point where I think everyone should have to go to some places to live where their computer doesn't have anything to do with it. at the same time, you need to go to a place they live where your computer might not be there.\nInput: i put in a lot of money, and we all get to use it for a purpose. We might be seeing a new car, but it is too expensive for us to invest in it and we have no money to spend.\ninput: i put in a lot of money, and we all get to use it for a purpose. We might be seeing a new car, but it is too expensive for us to invest in it and we have no money to invest. i put in a lot of money, and we all get to use it for a purpose. We might be seeing a new car, but it is too expensive for us to invest in it and we have no money to spend. i put in a lot of money, and we all get to use it for a purpose. We might be seeing a new car, but it is too expensive for us to invest in it and we have no money to spend. I have to keep my own money. I had 